# Librerías

Ejecutar esta celda siempre al comienzo. Si da error, reiniciar el entorno de ejecución.

In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

# Clone datasets
!apt-get install git
!git clone --depth=1 --filter=blob:none --sparse https://github.com/raulcastillabravo/datasets.git
%cd datasets
!git sparse-checkout set renfe/passengers/ renfe/stations/
%cd ..

import os
import sys
from datetime import datetime
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,032 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,648 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InR

# Esquemas

# Lectura

# Ejercicio 1

Escribe la tabla **passengers** particionada por la primera hora del tramo horario. Si **tramo_horario** es 09:00 - 09:30, la partición debe ser **tramo_inicio=0900**

**Funciones útiles**

```
F.split(F.col('colname'), ',')[0]  # divide los valores por coma y extrae el primero
F.regexp_replace(F.col('colname'), ',', ';')  # Cambia las comas por puntos y comas
```



# Ejercicio 2

Calcula el total de viajeros que han subido y bajado por núcleo de cercanías

**Funciones útiles**


```
F.sum(F.col('colname)).alias('new_colname') # Permite sumar los valores de una columna
```



# Ejercicio 3

Filtra los tramos horarios que tengan como **hora de inicio una hora en punto**. Si el tramo horario es 00:00 - 00:30, sí lo queremos, pero si es 00:30 - 01:00, no.



# Ejercicio 4

Calcula cuántos viajeros se suben a los trenes en cada población y ordena el resultado de mayor a menor.

**Pista**: tendrás que cruzar (**join**) la tabla de pasajeros con la de estaciones, agrupar y sumar. Puedes ordenar el resultado final usando **orderBy** de la siguiente forma


```
df.orderBy(F.col('colname'), ascending=False)  # Ordena de forma descendente
```



# Ejercicio 5 (micro proyecto)

Se ha detectado que en la tabla de estaciones hay fichas que están vacías (**NULL**). Se quiere evaluar el impacto de estas fichas vacías para saber a cuántos pasajeros les está afectando.

Para ello, se pide realizar un proceso automático que genere un informe con las siguientes características:

1. Debe mostrar la información agrupada por núcleo de cercanías.
2. Debe calcular dos KPIs:
  1. El porcentaje de viajeros que han subido a un tren sin ficha.
  2. El número de viajeros que han subido a un tren sin ficha.
3. El resultado debe estar ordenado de mayor a menor porcentaje.
4. El resultado debe escribirse particionado por núcleo de cercanías.
5. Cada vez que se ejecute el proceso, debe crearse una partición nueva con los resultados de la ejecución para poder tener trazabilidad de análisis anteriores.

**Pistas**

```
df.filter(F.col('colname').isNull())  # Filtra los valores NULL
df.filter(F.col('colname').isNotNull())  # Filtra los valores no NULL


# Permite crear una marca de tiempo con el instante de ejecucion
from datetime import datetime
now = datetime.now().strftime("%Y%m%d%H%M%S") # El resultado es un string
```



## Fase 1.

1. Cruzar tabla **df_passengers** con **df_station**.
2. Filtrar los casos donde las fichas son nulas y donde no lo son para crear dos DataFrames llamados **df_null** y **df_not_null**.

## Fase 2.

Agrupar cada DataFrame por separado para contar el número de viajeros subidos.

## Fase 3

Cruzar los resultados agregados y calcular el porcentaje de viajeros sin ficha

## Fase 4

Escribir el resultado final particionado por fecha de ejecución y núcleo de cercanías